In [25]:
import os
from random import sample
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Must be set before importing TF to supress messages
os.environ["CUDA_VISIBLE_DEVICES"]= '3'

import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger
import numpy as np
from utils.loader import DataLoader
from utils.tools import test_model, insert_attention_layer_in_keras
from utils.data_sampler import CustomDataGenerator, CustomIterator
from utils.configs import config
from typing import List
import pymf

def load_VGG_model(img_height: int, img_width: int, lr: int, loss: tf.keras.losses.Loss, metrics: List[str], trainable: True) -> tf.keras.Model:
    """ Loads VGG-16 model.

    Args:
        img_height (int): Image height.
        img_width (int): Image width.
        lr (int): Learning rate.
        loss (tf.keras.losses.Loss): Model loss.
        metrics (List[str]): Training metrics.
        trainable (True): Set if model weights should be kept frozen or not.

    Returns:
        tf.keras.Model: TensorFlow VGG-16 model.
    """
    model = tf.keras.applications.vgg16.VGG16(input_shape=(img_height, img_width, 3))
    model.trainable = trainable
    model.compile(optimizer=tf.keras.optimizers.Adam(lr, epsilon=0.1),
                loss=loss,
                metrics=metrics)

    return model

def train_model(model: tf.keras.Model, train_set: CustomIterator, val_set: CustomIterator, epochs: int, batch_size: int, callbacks=None):
    """ Train the model. 

    Args:
        train_set (CustomIterator): Training data.
        val_set (CustomIterator): Validation data.
        epochs (int): Number of epochs to train for.
        callbacks (_type_, optional): Callbacks for loggers. Defaults to None.

    Returns:
        history: Model training history information.
    """
    history = model.fit(train_set, validation_data=val_set, validation_freq=1, epochs=epochs, steps_per_epoch=train_set.n//batch_size, validation_steps=val_set.n//batch_size if val_set is not None else None, verbose=1, callbacks=callbacks)

    return history

- Ensure sampling correct during training
- Use same training params as Freddie paper
- Early stopping once validation loss increases

In [26]:
# Set configs
img_height = 224
img_width = 224
batch_size = 128
epochs = 1000
lr = 3e-4
log_path = os.path.join(config['logs_path'], 'vgg_baseline_attention_model_42.csv')

In [6]:
# Set augmentation and pre-processing
train_datagen = CustomDataGenerator(
                horizontal_flip=True,
                validation_split=0.1,
                preprocessing_function=tf.keras.applications.vgg16.preprocess_input, dtype=tf.float32)
test_datagen = CustomDataGenerator(
                preprocessing_function=tf.keras.applications.vgg16.preprocess_input, dtype=tf.float32)

# Load ImageNet dataset with the VGG augmentation
loader = DataLoader(batch_size, (img_height, img_width))
train_set = loader.load_train_set(aug_train=train_datagen, class_mode='categorical', shuffle=True)
val_set = loader.load_val_set(aug_val=train_datagen, class_mode='categorical', shuffle=True)
test_set = loader.load_test_set(aug_test=test_datagen, set_batch_size=False)

Loading train set...
Found 1153101 images belonging to 1000 classes.
Loading validation set...
Found 128066 images belonging to 1000 classes.
Loading test set...
Found 48238 images belonging to 1000 classes.


In [34]:
# Load pre-trained VGG-16 model
model = tf.keras.models.load_model('models/vgg_trained')
model.trainable = False

In [35]:
# Get layer kernel
kernel = model.get_layer('block4_conv3').kernel
flat_kernel = tf.reshape(kernel, [-1, kernel.shape[-1]]).numpy()

In [36]:
# Get projection matrix via SNMF
n_comp = 42

nmf = pymf.SNMF(flat_kernel, num_bases=n_comp)
nmf.factorize(niter=400)
p_mat = nmf.H

In [37]:
# Insert attention layer
model = insert_attention_layer_in_keras(p_mat, model, ['block5_conv1'])

In [38]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

In [12]:
# Enable subsampling equal to standard size
train_set.set_subsampling(size=1170*2, force_class_sampling=True)

In [39]:
# Train and use CSV logger to store logs
if not os.path.exists(log_path):
    with open(log_path, "w") as my_empty_csv: pass

csv_logger = CSVLogger(log_path, separator=',', append=False)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
train_history = train_model(model=model, train_set=train_set, val_set=val_set, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger, early_stop])

Epoch 1/1000
18/18 [==============================] - 643s 38s/step - loss: 0.7089 - accuracy: 0.8103 - val_loss: 0.7602 - val_accuracy: 0.7899
Epoch 2/1000
18/18 [==============================] - 646s 38s/step - loss: 0.6892 - accuracy: 0.8062 - val_loss: 0.7601 - val_accuracy: 0.7897
Epoch 3/1000
18/18 [==============================] - 650s 38s/step - loss: 0.6343 - accuracy: 0.8250 - val_loss: 0.7613 - val_accuracy: 0.7902
Epoch 4/1000
18/18 [==============================] - 651s 38s/step - loss: 0.7530 - accuracy: 0.7925 - val_loss: 0.7600 - val_accuracy: 0.7899
Epoch 5/1000
18/18 [==============================] - 658s 39s/step - loss: 0.6622 - accuracy: 0.8073 - val_loss: 0.7602 - val_accuracy: 0.7905
Epoch 6/1000
18/18 [==============================] - 642s 38s/step - loss: 0.6863 - accuracy: 0.8032 - val_loss: 0.7589 - val_accuracy: 0.7904
Epoch 7/1000
18/18 [==============================] - 643s 38s/step - loss: 0.6728 - accuracy: 0.8074 - val_loss: 0.7602 - val_accuracy:

Took 18, 10, 11, 10 for convergence

In [23]:
model.save_weights('models/baselines/baseline256')

Test all models

In [23]:
model.load_weights('models/baselines/baseline256')

In [24]:
test_model(model, test_set)


Predicting on test-set...
48238/48238 [==============================] - 234s 5ms/step
Computing accuracy...

-----------------------------------------
Model Accuracy on test-set: 0.6986608068327874
-----------------------------------------

